### For live streaming

In [1]:
import numpy as np
import cv2
import dlib
from PIL import Image
from skimage import io
import warnings
warnings.filterwarnings("ignore")

def detect_faces(image):
    """
        Using dlib library to extract faces from images, as the model was trained on faces.
        It would be better to use `dlib.cnn_face_detection_model_v1` but slower (http://dlib.net/cnn_face_detector.py.html)    
    """
    face_detector=dlib.get_frontal_face_detector()
    detected_faces=face_detector(image,1)
    face_frames=[(x.left(),x.top(),x.right(),x.bottom()) for x in detected_faces]
    
    #using cnn face detector
    #cnn_face_detector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
    # get `mmod_human_face_detector.dat` at http://dlib.net/files/mmod_human_face_detector.dat.bz2
    #detected_faces=cnn_face_detector(img,1)
    #face_frames=[(x.rect.left(),x.rect.top(),x.rect.right(),x.rect.bottom()) for x in detected_faces]
    
    return face_frames

def predict_on_image(model, in_image_path):
    """
        Makes prediction for imge(at `in_image_path`) and returns one of {"open", "close", "noface"}
        `model_path` - path to Keras model data (containing model layer data, model config data and weights)
    """
    
    def label_img(img, label, loc=(3,50)): #annotates the image with the predicted label (close/open)
        return cv2.putText(img, label, loc, cv2.FONT_HERSHEY_SIMPLEX, 3.4, (0, 255, 0), 2, cv2.LINE_AA)
    
    classes={1:"open",0:"close"}
    image=io.imread(in_image_path)
    face_rect=detect_faces(image)
    if len(face_rect)>0:
        image=np.array(Image.fromarray(image).crop(face_rect[0]))
    else:
        return "noface"
    # preprocess image - resize and remove mean from RGB channels
    processed_image=np.stack([cv2.resize(preprocess_input(img.astype(np.float32)),(224,224)) for img in [image]], axis=0)
    pred=model.predict(processed_image)    
    labels=[classes[p] for p in np.argmax(pred, axis=1)] #close/open label for image
    showStatistics(labels[0])
    return labels[0]


def showStatistics(label):

    textImage = np.zeros((300,512,3), np.uint8)
    className = ""
    cv2.putText(textImage,"Pedicted Class : " + label, 
    (30, 30), 
    cv2.FONT_HERSHEY_SIMPLEX, 
    1,
    (255, 255, 255),
    2)
    cv2.imshow("Statistics", textImage)

def main():
    model_path="best.h5"
    model=keras.models.load_model(model_path)

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)
    
    start_recording = False
    while(1):
        ret, img = camera.read()
    
        if start_recording:
            # display the frame with segmented hand
            cv2.imwrite('Temp_Eye_Grey.jpg',img)
            predict_on_image(model, 'Temp_Eye_Grey.jpg')
            
        cv2.imshow('img',img)
        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if user press q then it will stop looping
        if keypress == ord("q"):
            break
        #if user press s then it starts showing the o/p
        if keypress == ord("s"):
            start_recording = True
            
main()

### Predicting on Images

In [ ]:

def predict_on_image(model_path, in_image_path):
    """
        Makes prediction for imge(at `in_image_path`) and returns one of {"open", "close", "noface"}
        `model_path` - path to Keras model data (containing model layer data, model config data and weights)
    """
    
    def label_img(img, label, loc=(3,50)): #annotates the image with the predicted label (close/open)
        return cv2.putText(img, label, loc, cv2.FONT_HERSHEY_SIMPLEX, 3.4, (0, 255, 0), 2, cv2.LINE_AA)
    
    classes={1:"open",0:"close"}
    model=keras.models.load_model(model_path)
    image=io.imread(in_image_path)
    face_rect=detect_faces(image)
    if len(face_rect)>0:
        image=np.array(Image.fromarray(image).crop(face_rect[0]))
    else:
        return "noface"
    # preprocess image - resize and remove mean from RGB channels
    processed_image=np.stack([cv2.resize(preprocess_input(img.astype(np.float32)),(224,224)) for img in [image]], axis=0)
    pred=model.predict(processed_image)
    print(pred)
    labels=[classes[p] for p in np.argmax(pred, axis=1)] #close/open label for image
    return labels[0]

model_path="best.h5"
in_image_path="closed.jpg"#pass the name of your image or the path
predict_on_image(model_path, in_image_path)